In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import json
import glob
import sqlite3
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
with open('../parameters.json') as json_data :
    params = json.load(json_data)
#Constante souvent mise en majuscule=> nom de dossier, etc.
#on assigne les directory aux dossiers qu'on a créé

DATA_DIR = params['DIRECTORIES']['DATA_DIR']
TMP_DIR = params['DIRECTORIES']['TMP_DIR']    
PRICE_FILE = params['FILES']['PRICE_FILE']
STATION_FILE = params['FILES']['STATION_FILE']
SERVICE_FILE = params['FILES']['SERVICE_FILE']
#Autre méthode de lecture, pour éviter d'avoir à remettre os.path.join à chaque fois
GAS_DB = os.path.join(DATA_DIR,params['DATABASE']['GAS_DB']) 

In [3]:
# .. sert à remonter d'un niveau => esquive l'arborescence 
#Simple verif de ce qu'il y a dans la variable params
params

{'DATABASE': {'GAS_DB': 'gas.db'},
 'DIRECTORIES': {'DATA_DIR': '../../data', 'TMP_DIR': '../../tmp'},
 'FILES': {'PRICE_FILE': 'prices.p',
  'SERVICE_FILE': 'services.p',
  'STATION_FILE': 'stations.p'}}

In [4]:
#On va lire les données depuis le zip file mtnt
#on crée des listes 
#creation automatique de la liste des fichiers
gas_files = glob.glob(os.path.join(DATA_DIR,"Prix*.zip"))

station_files = 'Stations2016.zip'
service_files = 'Services2016.zip'

In [5]:
gas_files

['../../data\\Prix2014.zip',
 '../../data\\Prix2015.zip',
 '../../data\\Prix2016.zip']

In [6]:

dfall = pd.DataFrame()
for year in gas_files:
    df = pd.read_csv(year,
                     sep=';',header = None,dtype ={1:str} )
    df.columns = ['id_station','code_postal','type_station',
                     'latitude','longitude','date_releve',
                     'type_carburant','libelle_carburant',
                      'prix']
    dfall = pd.concat([dfall,df],axis=0)
dfall.shape
#Si le nombre de colonnes n'est pas égale à 9, on arrête
assert dfall.shape[1] == 9, 'Pas le bon nombre de colonnes'

In [7]:
dfall['date_releve'] = pd.to_datetime(dfall.date_releve)
dfall.head()

,id_station,code_postal,type_station,latitude,longitude,date_releve,type_carburant,libelle_carburant,prix
0,1000001,01000,R,4620114.0,519791.0,2014-01-02 11:08:03,1.0,Gazole,1304.0
1,1000001,01000,R,4620114.0,519791.0,2014-01-04 09:54:03,1.0,Gazole,1304.0
2,1000001,01000,R,4620114.0,519791.0,2014-01-05 10:27:09,1.0,Gazole,1304.0
3,1000001,01000,R,4620114.0,519791.0,2014-01-06 09:07:51,1.0,Gazole,1304.0
4,1000001,01000,R,4620114.0,519791.0,2014-01-07 09:23:56,1.0,Gazole,1304.0


In [8]:
df_stations = pd.read_csv(os.path.join(DATA_DIR,station_files),sep="|",
                          encoding='utf-8',
                          header = None,
                          dtype = {1:str})
df_stations.columns= ['id_stations','code_postal','type_station','latitude','longitude','adresse','ville']
df_stations.head()

,id_stations,code_postal,type_station,latitude,longitude,adresse,ville
0,1000001,01000,R,4.620114e+06,519791.000000,596 AVENUE DE TREVOUX,SAINT-DENIS-LèS-BOURG
1,1000002,01000,R,4.621842e+06,522767.000000,16 Avenue de Marboz,BOURG-EN-BRESSE
2,1000004,01000,R,4.618836e+06,524447.000000,20 Avenue du Maréchal Juin,Bourg-en-Bresse
3,1000005,01000,R,4.620094e+06,519942.024022,642 Avenue de Trévoux,SAINT-DENIS-LèS-BOURG
4,1000006,01000,R,4.620754e+06,523758.000000,1 Boulevard John Kennedy,BOURG-EN-BRESSE


In [9]:
df_services = pd.read_csv(os.path.join(DATA_DIR,service_files),sep="|",
                          encoding='utf-8',
                          header = None,
                          dtype = {1:str})
df_services.columns= ['id_station','code_postal','type_station','latitude','longitude','services']
df_services.head()

,id_station,code_postal,type_station,latitude,longitude,services
0,1000001,01000,R,4.620114e+06,519791.000000,"Station de gonflage,Vente de gaz domestique,Au..."
1,1000002,01000,R,4.621842e+06,522767.000000,"Vente de gaz domestique,Automate CB"
2,1000004,01000,R,4.618836e+06,524447.000000,"Carburant qualité supérieure,GPL,Toilettes pub..."
3,1000005,01000,R,4.620094e+06,519942.024022,"Carburant qualité supérieure,Station de lavage..."
4,1000006,01000,R,4.620754e+06,523758.000000,Vente de gaz domestique


In [10]:
dfall.to_pickle(os.path.join(DATA_DIR, PRICE_FILE))
df_stations.to_pickle(os.path.join(DATA_DIR, STATION_FILE))
df_services.to_pickle(os.path.join(DATA_DIR, SERVICE_FILE))

### FIN DU SCRIPT DE CHARGEMENT

In [ ]:
#transformation de la table en une base de donnée SQL

cnx = sqlite3.connect(GAS_DB)
dfall.to_sql('prices',cnx,index=False,if_exists='replace')
df_services.to_sql('services', cnx, index = False,if_exists='replace')
df_stations.to_sql('stations', cnx,index=False,if_exists='replace')



In [ ]:
cur = cnx.cursor()

In [ ]:
#création d'un curseur pour faire des requetes SQL
#On récupère le résultat de la requête dans res
cur.execute('SELECT * FROM prices where prix > 1400')
res = cur.fetchall()

In [ ]:
#solution plus rapide pour un dataframe
res_df=pd.read_sql_query("SELECT * FROM prices where prix>1400",cnx)

In [ ]:
res_df.head()

In [ ]:
dfall.head()

In [ ]:
#le .dt permet d'appeler les fonctions datetime sur la liste
dfall.date_releve.dt.week.head()

In [ ]:
#Si on est en 2014, la semaine 51 sera la semaine 51 mais si on est en 2015 
#On aura 51+52*1, on prendra les années en comptes
dfall['semaine'] =  dfall.date_releve.dt.week + 52* (dfall.date_releve.dt.year-2014)

In [ ]:
#On repère qu'il y a une erreur en 2016, il y avait 53 semaines en 2016
dfall[dfall.date_releve.dt.year==2016].date_releve.dt.week.max()

In [ ]:
dfall.groupby(['libelle_carburant','semaine']).agg({'prix':[np.mean,min,max]}).head()

In [ ]:
moyenne_hebdo_df =( dfall
                   .dropna(subset=['date_releve'])
                   .groupby(['libelle_carburant',pd.Grouper(key='date_releve',freq='2W-MON')])
                   .agg({'prix':[np.mean]})
                   .reset_index())

In [ ]:
moyenne_hebdo_df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [ ]:
graph  = sns.FacetGrid(moyenne_hebdo_df,hue='libelle_carburant',size=8,aspect=16/9)
graph.map(plt.plot,"date_releve","prix")
graph.add_legend()

# MODELISATION

In [ ]:
df_services.services.str.split(',').head()

In [ ]:
df_services['listes_services']=df_services.services.str.split(',')

In [ ]:
_tokenize = lambda x: x.split(',')

In [ ]:
vectorize = CountVectorizer(tokenizer = _tokenize, lowercase=False)

In [ ]:
#remplacement des valeurs manquantes pour éviter les erreurs
vectorize.fit_transform(df_services.services.fillna('Rien'))

In [ ]:
vectorize.vocabulary_